# import 및 drive 연결

In [1]:
# Google Drive와 연결
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 5.7 MB/s eta 0:00:00


In [3]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

# 모델
from lightgbm import LGBMClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# 한글 호환 문제
from matplotlib import font_manager as fm
from matplotlib import rc

!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

plt.rcParams['font.family'] = 'NanumBarunGothic'
font_path = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font_name = fm.FontProperties(fname=font_path).get_name()
plt.rcParams['font.family'] = font_name

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 35 not upgraded.
Need to get 10.3 MB of archives.
After this operation, 34.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fonts-nanum all 20200506-1 [10.3 MB]
Fetched 10.3 MB in 1s (12.3 MB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package fonts-nanum.
(Reading database ... 121749 files and direc

# 데이터 불러오기

In [4]:
research = pd.read_csv('/content/drive/MyDrive/3조(Rodi알려조)/9. 데이터셋/MBTI설문조사_ver1.csv')

In [5]:
research.columns

Index(['타임스탬프', '성별', '연령', '거주지', 'MBTI', '여행 선호도 [자연vs도시]',
       '여행 선호도 [숙박vs당일]', '여행 선호도 [익숙한지역(가본곳)vs새로운지역(안가본곳)]',
       '여행 선호도 [편하지만비싼숙소vs불편하지만저렴한숙소]', '여행 선호도 [휴양/휴식vs체험/활동]',
       '여행 선호도 [잘알려지지않은vs알려진]', '여행 선호도 [계획에 따른 vs 상황에 따른]',
       '여행 선호도 [사진촬영 중요하지않음vs사진촬영 중요]', '여행 동기 [1순위]', '여행 동기 [2순위]',
       '여행 동기 [3순위]', '좋았던 국내 여행지 1순위 시도', '좋았던  국내  여행지 1순위 세부',
       '좋았던 국내 여행지 2순위 시도', '좋았던 국내  여행지 2순위 세부', '좋았던 국내 여행지 3순위 시도',
       '좋았던 국내 여행지 3순위 시도.1', '이메일 주소'],
      dtype='object')

In [6]:
research.columns = ['타임스탬프', '성별', '연령', '거주지', 'MBTI', '자연/도시',
                    '숙박/당일', '익숙한/새로운',
                    '편하지만비싼숙소/불편하지만저렴한숙소', '휴양휴식/체험활동',
                    '잘알려지지않은/알려진', '계획에따른/상황에따른',
                    '사진촬영중요하지않음/사진촬영중요', '여행동기1순위', '여행동기2순위', '여행동기3순위',
                    '좋았던국내여행지 1순위 시도', '좋았던국내여행지 1순위 세부', '좋았던국내여행지 2순위 시도',
                    '좋았던국내여행지 2순위 세부', '좋았던국내여행지 3순위 시도', '좋았던국내여행지 3순위 세부', '이메일 주소']

In [7]:
research.head()

,타임스탬프,성별,연령,거주지,MBTI,자연/도시,숙박/당일,익숙한/새로운,편하지만비싼숙소/불편하지만저렴한숙소,휴양휴식/체험활동,...,여행동기1순위,여행동기2순위,여행동기3순위,좋았던국내여행지 1순위 시도,좋았던국내여행지 1순위 세부,좋았던국내여행지 2순위 시도,좋았던국내여행지 2순위 세부,좋았던국내여행지 3순위 시도,좋았던국내여행지 3순위 세부,이메일 주소
0,2024. 2. 7 오후 4:44:43,여자,20대,울산광역시,ENFP,후자 조금선호,전자 조금선호,후자 조금선호,전자 중간선호,후자 중간선호,...,"일상적인 환경 및 역할에서의 탈출, 지루함 탈피",새로운 경험 추구,여행 동반자와의 친밀감 및 유대감 증진,부산광역시,부산광역시,서울특별시,서울특별시,경상북도,경주시,NaN
1,2024. 2. 7 오후 4:45:59,여자,30대,충청남도,INFP,전자 중간선호,전자 조금선호,후자 매우선호,전자 중간선호,전자 조금선호,...,여행 동반자와의 친밀감 및 유대감 증진,진정한 자아 찾기 또는 자신을 되돌아볼 기회 찾기,새로운 경험 추구,강원도,강릉시,전라남도,여수시,경상남도,통영시,NaN
2,2024. 2. 7 오후 4:48:04,여자,20대,경기도,INTJ,전자 중간선호,전자 중간선호,후자 중간선호,전자 조금선호,후자 중간선호,...,"쉴 수 있는 기회, 육체 피로 해결 및 정신적인 휴식","일상적인 환경 및 역할에서의 탈출, 지루함 탈피",새로운 경험 추구,강원도,강릉시,제주특별자치도,서귀포시,부산광역시,부산광역시,NaN
3,2024. 2. 7 오후 4:48:08,여자,20대,울산광역시,ENFP,후자 조금선호,전자 조금선호,후자 조금선호,전자 중간선호,후자 중간선호,...,"일상적인 환경 및 역할에서의 탈출, 지루함 탈피",새로운 경험 추구,여행 동반자와의 친밀감 및 유대감 증진,부산광역시,부산광역시,서울특별시,서울특별시,경상북도,경주시,NaN
4,2024. 2. 7 오후 4:49:06,남자,20대,경기도,ENTP,전자 중간선호,전자 중간선호,후자 중간선호,전자 중간선호,전자 중간선호,...,"일상적인 환경 및 역할에서의 탈출, 지루함 탈피","쉴 수 있는 기회, 육체 피로 해결 및 정신적인 휴식",새로운 경험 추구,제주특별자치도,서귀포시,강원도,양양군,NaN,NaN,NaN


In [8]:
research_label = research.copy()
research_oh = research.copy()

# 데이터 전처리

## 여행스타일, 동기, 성별 라벨링

In [9]:
# 여행스타일
style_mapping = {
    '전자 매우선호': 1,
    '전자 중간선호': 2,
    '전자 조금선호': 3,
    '중립': 4,
    '후자 조금선호': 5,
    '후자 중간선호': 6,
    '후자 매우선호': 7,
}
style_columns = ['자연/도시',
       '숙박/당일', '익숙한/새로운',
       '편하지만비싼숙소/불편하지만저렴한숙소', '휴양휴식/체험활동',
       '잘알려지지않은/알려진', '계획에따른/상황에따른',
       '사진촬영중요하지않음/사진촬영중요']

for column in style_columns:
    research_label[column] = research_label[column].map(style_mapping)

In [10]:
# 여행동기
mtv_mapping = {'여행 동반자와의 친밀감 및 유대감 증진': 3,
           '새로운 경험 추구': 7,
           '쉴 수 있는 기회, 육체 피로 해결 및 정신적인 휴식': 2,
           '진정한 자아 찾기 또는 자신을 되돌아볼 기회 찾기': 4,
           '일상적인 환경 및 역할에서의 탈출, 지루함 탈피': 1,
           'SNS 사진 등록 등 과시': 5,
           '운동, 건강 증진 및 충전': 6,
           '역사 탐방, 문화적 경험 등 교육적 동기': 8,
           }
mtv_columns = ['여행동기1순위', '여행동기2순위', '여행동기3순위']

for column in mtv_columns:
    research_label[column] = research_label[column].map(mtv_mapping)

In [11]:
# 성별
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
research_label['성별'] = le.fit_transform(research_label['성별'])

In [12]:
research_label.head()

,타임스탬프,성별,연령,거주지,MBTI,자연/도시,숙박/당일,익숙한/새로운,편하지만비싼숙소/불편하지만저렴한숙소,휴양휴식/체험활동,...,여행동기1순위,여행동기2순위,여행동기3순위,좋았던국내여행지 1순위 시도,좋았던국내여행지 1순위 세부,좋았던국내여행지 2순위 시도,좋았던국내여행지 2순위 세부,좋았던국내여행지 3순위 시도,좋았던국내여행지 3순위 세부,이메일 주소
0,2024. 2. 7 오후 4:44:43,1,20대,울산광역시,ENFP,5,3,5,2,6,...,1,7,3,부산광역시,부산광역시,서울특별시,서울특별시,경상북도,경주시,NaN
1,2024. 2. 7 오후 4:45:59,1,30대,충청남도,INFP,2,3,7,2,3,...,3,4,7,강원도,강릉시,전라남도,여수시,경상남도,통영시,NaN
2,2024. 2. 7 오후 4:48:04,1,20대,경기도,INTJ,2,2,6,3,6,...,2,1,7,강원도,강릉시,제주특별자치도,서귀포시,부산광역시,부산광역시,NaN
3,2024. 2. 7 오후 4:48:08,1,20대,울산광역시,ENFP,5,3,5,2,6,...,1,7,3,부산광역시,부산광역시,서울특별시,서울특별시,경상북도,경주시,NaN
4,2024. 2. 7 오후 4:49:06,0,20대,경기도,ENTP,2,2,6,2,2,...,1,2,7,제주특별자치도,서귀포시,강원도,양양군,NaN,NaN,NaN


## 여행스타일, 동기, 성별 원핫인코딩

In [13]:
research_oh = pd.get_dummies(research_oh, columns=['성별', '자연/도시', '숙박/당일', '익숙한/새로운', '편하지만비싼숙소/불편하지만저렴한숙소',
                                    '휴양휴식/체험활동', '잘알려지지않은/알려진', '계획에따른/상황에따른', '사진촬영중요하지않음/사진촬영중요',
                                    '여행동기1순위', '여행동기2순위', '여행동기3순위',])
research_oh.head()

,타임스탬프,연령,거주지,MBTI,좋았던국내여행지 1순위 시도,좋았던국내여행지 1순위 세부,좋았던국내여행지 2순위 시도,좋았던국내여행지 2순위 세부,좋았던국내여행지 3순위 시도,좋았던국내여행지 3순위 세부,...,"여행동기2순위_일상적인 환경 및 역할에서의 탈출, 지루함 탈피",여행동기2순위_진정한 자아 찾기 또는 자신을 되돌아볼 기회 찾기,여행동기3순위_SNS 사진 등록 등 과시,여행동기3순위_새로운 경험 추구,"여행동기3순위_쉴 수 있는 기회, 육체 피로 해결 및 정신적인 휴식",여행동기3순위_여행 동반자와의 친밀감 및 유대감 증진,"여행동기3순위_역사 탐방, 문화적 경험 등 교육적 동기","여행동기3순위_운동, 건강 증진 및 충전","여행동기3순위_일상적인 환경 및 역할에서의 탈출, 지루함 탈피",여행동기3순위_진정한 자아 찾기 또는 자신을 되돌아볼 기회 찾기
0,2024. 2. 7 오후 4:44:43,20대,울산광역시,ENFP,부산광역시,부산광역시,서울특별시,서울특별시,경상북도,경주시,...,0,0,0,0,0,1,0,0,0,0
1,2024. 2. 7 오후 4:45:59,30대,충청남도,INFP,강원도,강릉시,전라남도,여수시,경상남도,통영시,...,0,1,0,1,0,0,0,0,0,0
2,2024. 2. 7 오후 4:48:04,20대,경기도,INTJ,강원도,강릉시,제주특별자치도,서귀포시,부산광역시,부산광역시,...,1,0,0,1,0,0,0,0,0,0
3,2024. 2. 7 오후 4:48:08,20대,울산광역시,ENFP,부산광역시,부산광역시,서울특별시,서울특별시,경상북도,경주시,...,0,0,0,0,0,1,0,0,0,0
4,2024. 2. 7 오후 4:49:06,20대,경기도,ENTP,제주특별자치도,서귀포시,강원도,양양군,NaN,NaN,...,0,0,0,1,0,0,0,0,0,0


## MBTI 요소별 column 추가

In [14]:
research_label['EI'] = research_label['MBTI'].str[0]
research_label['SN'] = research_label['MBTI'].str[1]
research_label['FT'] = research_label['MBTI'].str[2]
research_label['JP'] = research_label['MBTI'].str[3]

research_label.head()

,타임스탬프,성별,연령,거주지,MBTI,자연/도시,숙박/당일,익숙한/새로운,편하지만비싼숙소/불편하지만저렴한숙소,휴양휴식/체험활동,...,좋았던국내여행지 1순위 세부,좋았던국내여행지 2순위 시도,좋았던국내여행지 2순위 세부,좋았던국내여행지 3순위 시도,좋았던국내여행지 3순위 세부,이메일 주소,EI,SN,FT,JP
0,2024. 2. 7 오후 4:44:43,1,20대,울산광역시,ENFP,5,3,5,2,6,...,부산광역시,서울특별시,서울특별시,경상북도,경주시,NaN,E,N,F,P
1,2024. 2. 7 오후 4:45:59,1,30대,충청남도,INFP,2,3,7,2,3,...,강릉시,전라남도,여수시,경상남도,통영시,NaN,I,N,F,P
2,2024. 2. 7 오후 4:48:04,1,20대,경기도,INTJ,2,2,6,3,6,...,강릉시,제주특별자치도,서귀포시,부산광역시,부산광역시,NaN,I,N,T,J
3,2024. 2. 7 오후 4:48:08,1,20대,울산광역시,ENFP,5,3,5,2,6,...,부산광역시,서울특별시,서울특별시,경상북도,경주시,NaN,E,N,F,P
4,2024. 2. 7 오후 4:49:06,0,20대,경기도,ENTP,2,2,6,2,2,...,서귀포시,강원도,양양군,NaN,NaN,NaN,E,N,T,P


In [15]:
research_oh['EI'] = research_oh['MBTI'].str[0]
research_oh['SN'] = research_oh['MBTI'].str[1]
research_oh['FT'] = research_oh['MBTI'].str[2]
research_oh['JP'] = research_oh['MBTI'].str[3]

research_oh.head()

,타임스탬프,연령,거주지,MBTI,좋았던국내여행지 1순위 시도,좋았던국내여행지 1순위 세부,좋았던국내여행지 2순위 시도,좋았던국내여행지 2순위 세부,좋았던국내여행지 3순위 시도,좋았던국내여행지 3순위 세부,...,"여행동기3순위_쉴 수 있는 기회, 육체 피로 해결 및 정신적인 휴식",여행동기3순위_여행 동반자와의 친밀감 및 유대감 증진,"여행동기3순위_역사 탐방, 문화적 경험 등 교육적 동기","여행동기3순위_운동, 건강 증진 및 충전","여행동기3순위_일상적인 환경 및 역할에서의 탈출, 지루함 탈피",여행동기3순위_진정한 자아 찾기 또는 자신을 되돌아볼 기회 찾기,EI,SN,FT,JP
0,2024. 2. 7 오후 4:44:43,20대,울산광역시,ENFP,부산광역시,부산광역시,서울특별시,서울특별시,경상북도,경주시,...,0,1,0,0,0,0,E,N,F,P
1,2024. 2. 7 오후 4:45:59,30대,충청남도,INFP,강원도,강릉시,전라남도,여수시,경상남도,통영시,...,0,0,0,0,0,0,I,N,F,P
2,2024. 2. 7 오후 4:48:04,20대,경기도,INTJ,강원도,강릉시,제주특별자치도,서귀포시,부산광역시,부산광역시,...,0,0,0,0,0,0,I,N,T,J
3,2024. 2. 7 오후 4:48:08,20대,울산광역시,ENFP,부산광역시,부산광역시,서울특별시,서울특별시,경상북도,경주시,...,0,1,0,0,0,0,E,N,F,P
4,2024. 2. 7 오후 4:49:06,20대,경기도,ENTP,제주특별자치도,서귀포시,강원도,양양군,NaN,NaN,...,0,0,0,0,0,0,E,N,T,P


## 여행객 성별O/X, 동기O/X로 분리(정확도 기준)

In [16]:
# 성별 X, 동기 X, 원핫인코딩
mbti = research_oh[['MBTI', '자연/도시_전자 매우선호',
       '자연/도시_전자 조금선호', '자연/도시_전자 중간선호', '자연/도시_후자 매우선호', '자연/도시_후자 조금선호',
       '자연/도시_후자 중간선호', '숙박/당일_전자 매우선호', '숙박/당일_전자 조금선호', '숙박/당일_전자 중간선호',
       '숙박/당일_후자 매우선호', '숙박/당일_후자 조금선호', '숙박/당일_후자 중간선호', '익숙한/새로운_전자 매우선호',
       '익숙한/새로운_전자 조금선호', '익숙한/새로운_전자 중간선호', '익숙한/새로운_후자 매우선호',
       '익숙한/새로운_후자 조금선호', '익숙한/새로운_후자 중간선호', '편하지만비싼숙소/불편하지만저렴한숙소_전자 매우선호',
       '편하지만비싼숙소/불편하지만저렴한숙소_전자 조금선호', '편하지만비싼숙소/불편하지만저렴한숙소_전자 중간선호',
       '편하지만비싼숙소/불편하지만저렴한숙소_후자 매우선호', '편하지만비싼숙소/불편하지만저렴한숙소_후자 조금선호',
       '편하지만비싼숙소/불편하지만저렴한숙소_후자 중간선호', '휴양휴식/체험활동_전자 매우선호', '휴양휴식/체험활동_전자 조금선호',
       '휴양휴식/체험활동_전자 중간선호', '휴양휴식/체험활동_후자 매우선호', '휴양휴식/체험활동_후자 조금선호',
       '휴양휴식/체험활동_후자 중간선호', '잘알려지지않은/알려진_전자 매우선호', '잘알려지지않은/알려진_전자 조금선호',
       '잘알려지지않은/알려진_전자 중간선호', '잘알려지지않은/알려진_후자 매우선호', '잘알려지지않은/알려진_후자 조금선호',
       '잘알려지지않은/알려진_후자 중간선호', '계획에따른/상황에따른_전자 매우선호', '계획에따른/상황에따른_전자 조금선호',
       '계획에따른/상황에따른_전자 중간선호', '계획에따른/상황에따른_후자 매우선호', '계획에따른/상황에따른_후자 조금선호',
       '계획에따른/상황에따른_후자 중간선호', '사진촬영중요하지않음/사진촬영중요_전자 매우선호',
       '사진촬영중요하지않음/사진촬영중요_전자 조금선호', '사진촬영중요하지않음/사진촬영중요_전자 중간선호',
       '사진촬영중요하지않음/사진촬영중요_후자 매우선호', '사진촬영중요하지않음/사진촬영중요_후자 조금선호',
       '사진촬영중요하지않음/사진촬영중요_후자 중간선호',
       'EI', 'SN', 'FT', 'JP']]
mbti.head()

,MBTI,자연/도시_전자 매우선호,자연/도시_전자 조금선호,자연/도시_전자 중간선호,자연/도시_후자 매우선호,자연/도시_후자 조금선호,자연/도시_후자 중간선호,숙박/당일_전자 매우선호,숙박/당일_전자 조금선호,숙박/당일_전자 중간선호,...,사진촬영중요하지않음/사진촬영중요_전자 매우선호,사진촬영중요하지않음/사진촬영중요_전자 조금선호,사진촬영중요하지않음/사진촬영중요_전자 중간선호,사진촬영중요하지않음/사진촬영중요_후자 매우선호,사진촬영중요하지않음/사진촬영중요_후자 조금선호,사진촬영중요하지않음/사진촬영중요_후자 중간선호,EI,SN,FT,JP
0,ENFP,0,0,0,0,1,0,0,1,0,...,0,0,0,1,0,0,E,N,F,P
1,INFP,0,0,1,0,0,0,0,1,0,...,0,0,1,0,0,0,I,N,F,P
2,INTJ,0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,1,I,N,T,J
3,ENFP,0,0,0,0,1,0,0,1,0,...,0,0,0,1,0,0,E,N,F,P
4,ENTP,0,0,1,0,0,0,0,0,1,...,0,0,0,0,1,0,E,N,T,P


In [17]:
# 성별 O, 동기 X, 라벨링
mbti_gender = research_label[['MBTI', '성별', '자연/도시', '숙박/당일', '익숙한/새로운', '편하지만비싼숙소/불편하지만저렴한숙소',
                        '휴양휴식/체험활동', '잘알려지지않은/알려진', '계획에따른/상황에따른', '사진촬영중요하지않음/사진촬영중요',
                        'EI', 'SN', 'FT', 'JP']]
mbti_gender.head()

,MBTI,성별,자연/도시,숙박/당일,익숙한/새로운,편하지만비싼숙소/불편하지만저렴한숙소,휴양휴식/체험활동,잘알려지지않은/알려진,계획에따른/상황에따른,사진촬영중요하지않음/사진촬영중요,EI,SN,FT,JP
0,ENFP,1,5,3,5,2,6,5,3,7,E,N,F,P
1,INFP,1,2,3,7,2,3,1,2,2,I,N,F,P
2,INTJ,1,2,2,6,3,6,5,6,6,I,N,T,J
3,ENFP,1,5,3,5,2,6,5,3,7,E,N,F,P
4,ENTP,0,2,2,6,2,2,6,6,5,E,N,T,P


In [18]:
# 성별 X, 동기 O, 원핫인코딩
mbti_motivation = research_oh[['MBTI', '여행동기1순위_새로운 경험 추구',
       '여행동기1순위_쉴 수 있는 기회, 육체 피로 해결 및 정신적인 휴식',
       '여행동기1순위_여행 동반자와의 친밀감 및 유대감 증진', '여행동기1순위_역사 탐방, 문화적 경험 등 교육적 동기',
       '여행동기1순위_일상적인 환경 및 역할에서의 탈출, 지루함 탈피',
       '여행동기1순위_진정한 자아 찾기 또는 자신을 되돌아볼 기회 찾기', '여행동기2순위_새로운 경험 추구',
       '여행동기2순위_쉴 수 있는 기회, 육체 피로 해결 및 정신적인 휴식',
       '여행동기2순위_여행 동반자와의 친밀감 및 유대감 증진', '여행동기2순위_역사 탐방, 문화적 경험 등 교육적 동기',
       '여행동기2순위_운동, 건강 증진 및 충전', '여행동기2순위_일상적인 환경 및 역할에서의 탈출, 지루함 탈피',
       '여행동기2순위_진정한 자아 찾기 또는 자신을 되돌아볼 기회 찾기', '여행동기3순위_SNS 사진 등록 등 과시',
       '여행동기3순위_새로운 경험 추구', '여행동기3순위_쉴 수 있는 기회, 육체 피로 해결 및 정신적인 휴식',
       '여행동기3순위_여행 동반자와의 친밀감 및 유대감 증진', '여행동기3순위_역사 탐방, 문화적 경험 등 교육적 동기',
       '여행동기3순위_운동, 건강 증진 및 충전', '여행동기3순위_일상적인 환경 및 역할에서의 탈출, 지루함 탈피',
       '여행동기3순위_진정한 자아 찾기 또는 자신을 되돌아볼 기회 찾기', '자연/도시_전자 매우선호',
       '자연/도시_전자 조금선호', '자연/도시_전자 중간선호', '자연/도시_후자 매우선호', '자연/도시_후자 조금선호',
       '자연/도시_후자 중간선호', '숙박/당일_전자 매우선호', '숙박/당일_전자 조금선호', '숙박/당일_전자 중간선호',
       '숙박/당일_후자 매우선호', '숙박/당일_후자 조금선호', '숙박/당일_후자 중간선호', '익숙한/새로운_전자 매우선호',
       '익숙한/새로운_전자 조금선호', '익숙한/새로운_전자 중간선호', '익숙한/새로운_후자 매우선호',
       '익숙한/새로운_후자 조금선호', '익숙한/새로운_후자 중간선호', '편하지만비싼숙소/불편하지만저렴한숙소_전자 매우선호',
       '편하지만비싼숙소/불편하지만저렴한숙소_전자 조금선호', '편하지만비싼숙소/불편하지만저렴한숙소_전자 중간선호',
       '편하지만비싼숙소/불편하지만저렴한숙소_후자 매우선호', '편하지만비싼숙소/불편하지만저렴한숙소_후자 조금선호',
       '편하지만비싼숙소/불편하지만저렴한숙소_후자 중간선호', '휴양휴식/체험활동_전자 매우선호', '휴양휴식/체험활동_전자 조금선호',
       '휴양휴식/체험활동_전자 중간선호', '휴양휴식/체험활동_후자 매우선호', '휴양휴식/체험활동_후자 조금선호',
       '휴양휴식/체험활동_후자 중간선호', '잘알려지지않은/알려진_전자 매우선호', '잘알려지지않은/알려진_전자 조금선호',
       '잘알려지지않은/알려진_전자 중간선호', '잘알려지지않은/알려진_후자 매우선호', '잘알려지지않은/알려진_후자 조금선호',
       '잘알려지지않은/알려진_후자 중간선호', '계획에따른/상황에따른_전자 매우선호', '계획에따른/상황에따른_전자 조금선호',
       '계획에따른/상황에따른_전자 중간선호', '계획에따른/상황에따른_후자 매우선호', '계획에따른/상황에따른_후자 조금선호',
       '계획에따른/상황에따른_후자 중간선호', '사진촬영중요하지않음/사진촬영중요_전자 매우선호',
       '사진촬영중요하지않음/사진촬영중요_전자 조금선호', '사진촬영중요하지않음/사진촬영중요_전자 중간선호',
       '사진촬영중요하지않음/사진촬영중요_후자 매우선호', '사진촬영중요하지않음/사진촬영중요_후자 조금선호',
       '사진촬영중요하지않음/사진촬영중요_후자 중간선호',
       'EI', 'SN', 'FT', 'JP']]
mbti_motivation.head()

,MBTI,여행동기1순위_새로운 경험 추구,"여행동기1순위_쉴 수 있는 기회, 육체 피로 해결 및 정신적인 휴식",여행동기1순위_여행 동반자와의 친밀감 및 유대감 증진,"여행동기1순위_역사 탐방, 문화적 경험 등 교육적 동기","여행동기1순위_일상적인 환경 및 역할에서의 탈출, 지루함 탈피",여행동기1순위_진정한 자아 찾기 또는 자신을 되돌아볼 기회 찾기,여행동기2순위_새로운 경험 추구,"여행동기2순위_쉴 수 있는 기회, 육체 피로 해결 및 정신적인 휴식",여행동기2순위_여행 동반자와의 친밀감 및 유대감 증진,...,사진촬영중요하지않음/사진촬영중요_전자 매우선호,사진촬영중요하지않음/사진촬영중요_전자 조금선호,사진촬영중요하지않음/사진촬영중요_전자 중간선호,사진촬영중요하지않음/사진촬영중요_후자 매우선호,사진촬영중요하지않음/사진촬영중요_후자 조금선호,사진촬영중요하지않음/사진촬영중요_후자 중간선호,EI,SN,FT,JP
0,ENFP,0,0,0,0,1,0,1,0,0,...,0,0,0,1,0,0,E,N,F,P
1,INFP,0,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,I,N,F,P
2,INTJ,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,I,N,T,J
3,ENFP,0,0,0,0,1,0,1,0,0,...,0,0,0,1,0,0,E,N,F,P
4,ENTP,0,0,0,0,1,0,0,1,0,...,0,0,0,0,1,0,E,N,T,P


In [19]:
# 성별 O, 동기 O, 라벨링
mbti_all = research_label[['MBTI', '성별', '여행동기1순위', '여행동기2순위', '여행동기3순위', '자연/도시', '숙박/당일', '익숙한/새로운', '편하지만비싼숙소/불편하지만저렴한숙소',
                     '휴양휴식/체험활동', '잘알려지지않은/알려진', '계획에따른/상황에따른', '사진촬영중요하지않음/사진촬영중요',
                     'EI', 'SN', 'FT', 'JP']]
mbti_all.head()

,MBTI,성별,여행동기1순위,여행동기2순위,여행동기3순위,자연/도시,숙박/당일,익숙한/새로운,편하지만비싼숙소/불편하지만저렴한숙소,휴양휴식/체험활동,잘알려지지않은/알려진,계획에따른/상황에따른,사진촬영중요하지않음/사진촬영중요,EI,SN,FT,JP
0,ENFP,1,1,7,3,5,3,5,2,6,5,3,7,E,N,F,P
1,INFP,1,3,4,7,2,3,7,2,3,1,2,2,I,N,F,P
2,INTJ,1,2,1,7,2,2,6,3,6,5,6,6,I,N,T,J
3,ENFP,1,1,7,3,5,3,5,2,6,5,3,7,E,N,F,P
4,ENTP,0,1,2,7,2,2,6,2,2,6,6,5,E,N,T,P


# 데이터셋 분리

In [20]:
# 성별 X, 동기 O, 원핫, EI 분리
X_EI_motivation = mbti_motivation.iloc[:, 1:-4]
y_EI_motivation = mbti_motivation.iloc[:, -4]
X_train_EI_motivation, X_test_EI_motivation, y_train_EI_motivation, y_test_EI_motivation = train_test_split(X_EI_motivation, y_EI_motivation, test_size=0.2, random_state=0)

In [21]:
# 성별 O, 동기 O, 라벨링, SN 분리
X_SN_all = mbti_all.iloc[:, 1:-4]
y_SN_all = mbti_all.iloc[:, -3]
X_train_SN_all, X_test_SN_all, y_train_SN_all, y_test_SN_all = train_test_split(X_SN_all, y_SN_all, test_size=0.2, random_state=0)

In [22]:
# 성별 X, 동기 X, 원핫, FT 분리
X_FT = mbti.iloc[:, 1:-4]
y_FT = mbti.iloc[:, -2]
X_train_FT, X_test_FT, y_train_FT, y_test_FT = train_test_split(X_FT, y_FT, test_size=0.2, random_state=0)

In [23]:
# 성별 O, 동기 X, 라벨링, JP 분리
X_JP_gender = mbti_gender.iloc[:, 1:-4]
y_JP_gender = mbti_gender.iloc[:, -1]
X_train_JP_gender, X_test_JP_gender, y_train_JP_gender, y_test_JP_gender = train_test_split(X_JP_gender, y_JP_gender, test_size=0.2, random_state=0)

# MBTI 예측모델

In [24]:
# E/I, 성별 X, 동기 O, 원핫, DT
dt_clf_EI_motivation = DecisionTreeClassifier(random_state=0)
dt_clf_EI_motivation.fit(X_train_EI_motivation, y_train_EI_motivation)
pred_EI_motivation = dt_clf_EI_motivation.predict(X_test_EI_motivation)

In [25]:
# S/N, 성별 O, 동기 O, 라벨링, CB
cb_clf_SN_all = CatBoostClassifier(random_state=0)
cb_clf_SN_all.fit(X_train_SN_all, y_train_SN_all)
pred_SN_all = cb_clf_SN_all.predict(X_test_SN_all)

Learning rate set to 0.004045
0:	learn: 0.6917174	total: 50.5ms	remaining: 50.4s
1:	learn: 0.6907943	total: 51.5ms	remaining: 25.7s
2:	learn: 0.6891392	total: 53.7ms	remaining: 17.8s
3:	learn: 0.6881452	total: 54.8ms	remaining: 13.7s
4:	learn: 0.6863636	total: 55.8ms	remaining: 11.1s
5:	learn: 0.6846937	total: 56.8ms	remaining: 9.4s
6:	learn: 0.6834876	total: 57.8ms	remaining: 8.2s
7:	learn: 0.6823330	total: 58.4ms	remaining: 7.24s
8:	learn: 0.6809940	total: 59.4ms	remaining: 6.54s
9:	learn: 0.6797890	total: 60.3ms	remaining: 5.97s
10:	learn: 0.6786463	total: 61.1ms	remaining: 5.5s
11:	learn: 0.6776081	total: 62ms	remaining: 5.1s
12:	learn: 0.6762090	total: 62.8ms	remaining: 4.77s
13:	learn: 0.6748837	total: 63.8ms	remaining: 4.49s
14:	learn: 0.6734156	total: 64.6ms	remaining: 4.24s
15:	learn: 0.6727406	total: 65.5ms	remaining: 4.03s
16:	learn: 0.6716849	total: 66.3ms	remaining: 3.83s
17:	learn: 0.6705493	total: 67.2ms	remaining: 3.66s
18:	learn: 0.6693117	total: 68ms	remaining: 3.51s


In [26]:
# F/T, 성별 X, 동기 X, 원핫, RLM
rf_clf_FT = RandomForestClassifier(random_state=0)
rf_clf_FT.fit(X_train_FT, y_train_FT)
pred_FT = rf_clf_FT.predict(X_test_FT)

In [27]:
# J/P, 성별 O, 동기 X, 라벨링, CB
cb_clf_JP_gender = CatBoostClassifier(random_state=0)
cb_clf_JP_gender.fit(X_train_JP_gender, y_train_JP_gender)
pred_JP_gender = cb_clf_JP_gender.predict(X_test_JP_gender)

Learning rate set to 0.004045
0:	learn: 0.6909228	total: 956us	remaining: 955ms
1:	learn: 0.6888509	total: 1.83ms	remaining: 914ms
2:	learn: 0.6872187	total: 2.68ms	remaining: 890ms
3:	learn: 0.6856616	total: 3.62ms	remaining: 902ms
4:	learn: 0.6833263	total: 4.7ms	remaining: 934ms
5:	learn: 0.6822867	total: 5.41ms	remaining: 897ms
6:	learn: 0.6802652	total: 6.29ms	remaining: 893ms
7:	learn: 0.6785175	total: 7.07ms	remaining: 877ms
8:	learn: 0.6768731	total: 7.85ms	remaining: 864ms
9:	learn: 0.6750683	total: 8.65ms	remaining: 856ms
10:	learn: 0.6727651	total: 9.46ms	remaining: 850ms
11:	learn: 0.6709386	total: 10.3ms	remaining: 851ms
12:	learn: 0.6688980	total: 11.1ms	remaining: 846ms
13:	learn: 0.6667723	total: 12ms	remaining: 845ms
14:	learn: 0.6645609	total: 12.9ms	remaining: 845ms
15:	learn: 0.6636362	total: 13.7ms	remaining: 840ms
16:	learn: 0.6621697	total: 14.4ms	remaining: 834ms
17:	learn: 0.6600701	total: 15.2ms	remaining: 831ms
18:	learn: 0.6582925	total: 16ms	remaining: 827m

# 설문조사 test셋 MBTI 예측

In [37]:
research_test = research.iloc[X_test_FT.index]
research_test.drop(['타임스탬프', '이메일 주소'], axis=1, inplace=True)
research_test

,성별,연령,거주지,MBTI,자연/도시,숙박/당일,익숙한/새로운,편하지만비싼숙소/불편하지만저렴한숙소,휴양휴식/체험활동,잘알려지지않은/알려진,...,사진촬영중요하지않음/사진촬영중요,여행동기1순위,여행동기2순위,여행동기3순위,좋았던국내여행지 1순위 시도,좋았던국내여행지 1순위 세부,좋았던국내여행지 2순위 시도,좋았던국내여행지 2순위 세부,좋았던국내여행지 3순위 시도,좋았던국내여행지 3순위 세부
45,남자,20대,경상남도,INTJ,후자 중간선호,전자 중간선호,후자 매우선호,후자 조금선호,후자 중간선호,후자 조금선호,...,후자 중간선호,"일상적인 환경 및 역할에서의 탈출, 지루함 탈피","쉴 수 있는 기회, 육체 피로 해결 및 정신적인 휴식",새로운 경험 추구,제주특별자치도,서귀포시,강원도,태백시,전라남도,남해군
60,남자,30대,제주특별자치도,ISFJ,전자 중간선호,전자 매우선호,전자 조금선호,전자 조금선호,전자 중간선호,전자 조금선호,...,전자 매우선호,"쉴 수 있는 기회, 육체 피로 해결 및 정신적인 휴식","일상적인 환경 및 역할에서의 탈출, 지루함 탈피",진정한 자아 찾기 또는 자신을 되돌아볼 기회 찾기,전라남도,담양,제주특별자치도,제주시,NaN,NaN
7,여자,30대,서울특별시,ESTP,후자 매우선호,전자 매우선호,후자 매우선호,전자 매우선호,후자 매우선호,전자 매우선호,...,후자 매우선호,새로운 경험 추구,"일상적인 환경 및 역할에서의 탈출, 지루함 탈피","쉴 수 있는 기회, 육체 피로 해결 및 정신적인 휴식",제주특별자치도,제주시,강원도,강릉시,부산광역시,부산광역시
51,남자,20대,전라북도,ENTJ,후자 조금선호,전자 조금선호,후자 중간선호,전자 조금선호,후자 조금선호,후자 조금선호,...,전자 중간선호,진정한 자아 찾기 또는 자신을 되돌아볼 기회 찾기,새로운 경험 추구,"일상적인 환경 및 역할에서의 탈출, 지루함 탈피",제주특별자치도,제주도,강원도,강릉,서울특별시,송파구
66,여자,30대,울산광역시,ENFP,전자 중간선호,전자 매우선호,후자 매우선호,후자 중간선호,후자 매우선호,후자 중간선호,...,전자 중간선호,새로운 경험 추구,"일상적인 환경 및 역할에서의 탈출, 지루함 탈피",여행 동반자와의 친밀감 및 유대감 증진,경상북도,통영시,경상남도,하동시,강원도,NaN
27,남자,20대,제주특별자치도,ISFJ,후자 중간선호,전자 매우선호,후자 매우선호,전자 매우선호,전자 매우선호,후자 매우선호,...,후자 매우선호,"일상적인 환경 및 역할에서의 탈출, 지루함 탈피",여행 동반자와의 친밀감 및 유대감 증진,새로운 경험 추구,서울특별시,홍대,부산광역시,서먄,제주특별자치도,서귀포시
71,여자,20대,제주특별자치도,INFP,후자 중간선호,후자 중간선호,후자 매우선호,후자 중간선호,전자 중간선호,후자 중간선호,...,후자 중간선호,"쉴 수 있는 기회, 육체 피로 해결 및 정신적인 휴식","일상적인 환경 및 역할에서의 탈출, 지루함 탈피",새로운 경험 추구,경상북도,경주시,서울특별시,서울시,NaN,NaN
54,여자,30대,서울특별시,ISTJ,전자 조금선호,전자 중간선호,후자 중간선호,전자 중간선호,후자 중간선호,전자 중간선호,...,전자 중간선호,"일상적인 환경 및 역할에서의 탈출, 지루함 탈피","쉴 수 있는 기회, 육체 피로 해결 및 정신적인 휴식",여행 동반자와의 친밀감 및 유대감 증진,제주특별자치도,제주도,강원도,강릉,NaN,NaN
123,여자,30대,서울특별시,ISFP,전자 중간선호,전자 매우선호,후자 조금선호,후자 조금선호,전자 매우선호,전자 매우선호,...,전자 조금선호,"일상적인 환경 및 역할에서의 탈출, 지루함 탈피","쉴 수 있는 기회, 육체 피로 해결 및 정신적인 휴식",새로운 경험 추구,부산광역시,부산시,전라북도,남원시,제주특별자치도,서귀포시
8,남자,20대,서울특별시,INFJ,후자 매우선호,후자 조금선호,전자 매우선호,전자 조금선호,후자 매우선호,전자 매우선호,...,후자 중간선호,"역사 탐방, 문화적 경험 등 교육적 동기",새로운 경험 추구,진정한 자아 찾기 또는 자신을 되돌아볼 기회 찾기,경상북도,경주시,서울특별시,경복궁,서울특별시,화계사


In [42]:
research_test['EI'] = pred_EI_motivation
research_test['SN'] = pred_SN_all
research_test['FT'] = pred_FT
research_test['JP'] = pred_JP_gender
research_test['MBTI예측값'] = research_test['EI'] + research_test['SN'] + research_test['FT'] + research_test['JP']
research_test.rename({'MBTI':'MBTI실제값'}, inplace=True, axis=1)
research_test

,성별,연령,거주지,MBTI실제값,자연/도시,숙박/당일,익숙한/새로운,편하지만비싼숙소/불편하지만저렴한숙소,휴양휴식/체험활동,잘알려지지않은/알려진,...,좋았던국내여행지 1순위 세부,좋았던국내여행지 2순위 시도,좋았던국내여행지 2순위 세부,좋았던국내여행지 3순위 시도,좋았던국내여행지 3순위 세부,EI,SN,FT,JP,MBTI예측값
45,남자,20대,경상남도,INTJ,후자 중간선호,전자 중간선호,후자 매우선호,후자 조금선호,후자 중간선호,후자 조금선호,...,서귀포시,강원도,태백시,전라남도,남해군,I,S,F,J,ISFJ
60,남자,30대,제주특별자치도,ISFJ,전자 중간선호,전자 매우선호,전자 조금선호,전자 조금선호,전자 중간선호,전자 조금선호,...,담양,제주특별자치도,제주시,NaN,NaN,I,S,F,J,ISFJ
7,여자,30대,서울특별시,ESTP,후자 매우선호,전자 매우선호,후자 매우선호,전자 매우선호,후자 매우선호,전자 매우선호,...,제주시,강원도,강릉시,부산광역시,부산광역시,E,N,F,P,ENFP
51,남자,20대,전라북도,ENTJ,후자 조금선호,전자 조금선호,후자 중간선호,전자 조금선호,후자 조금선호,후자 조금선호,...,제주도,강원도,강릉,서울특별시,송파구,E,N,F,J,ENFJ
66,여자,30대,울산광역시,ENFP,전자 중간선호,전자 매우선호,후자 매우선호,후자 중간선호,후자 매우선호,후자 중간선호,...,통영시,경상남도,하동시,강원도,NaN,I,S,F,P,ISFP
27,남자,20대,제주특별자치도,ISFJ,후자 중간선호,전자 매우선호,후자 매우선호,전자 매우선호,전자 매우선호,후자 매우선호,...,홍대,부산광역시,서먄,제주특별자치도,서귀포시,I,S,F,J,ISFJ
71,여자,20대,제주특별자치도,INFP,후자 중간선호,후자 중간선호,후자 매우선호,후자 중간선호,전자 중간선호,후자 중간선호,...,경주시,서울특별시,서울시,NaN,NaN,I,N,F,P,INFP
54,여자,30대,서울특별시,ISTJ,전자 조금선호,전자 중간선호,후자 중간선호,전자 중간선호,후자 중간선호,전자 중간선호,...,제주도,강원도,강릉,NaN,NaN,I,S,F,J,ISFJ
123,여자,30대,서울특별시,ISFP,전자 중간선호,전자 매우선호,후자 조금선호,후자 조금선호,전자 매우선호,전자 매우선호,...,부산시,전라북도,남원시,제주특별자치도,서귀포시,I,N,F,P,INFP
8,남자,20대,서울특별시,INFJ,후자 매우선호,후자 조금선호,전자 매우선호,전자 조금선호,후자 매우선호,전자 매우선호,...,경주시,서울특별시,경복궁,서울특별시,화계사,E,N,F,J,ENFJ


In [41]:
research_test_summary = research_test[['MBTI실제값', 'MBTI예측값']]
research_test_summary

,MBTI실제값,MBTI예측값
45,INTJ,ISFJ
60,ISFJ,ISFJ
7,ESTP,ENFP
51,ENTJ,ENFJ
66,ENFP,ISFP
27,ISFJ,ISFJ
71,INFP,INFP
54,ISTJ,ISFJ
123,ISFP,INFP
8,INFJ,ENFJ


In [43]:
accuracy_research_test = accuracy_score(research_test_summary['MBTI실제값'], research_test_summary['MBTI예측값'])
accuracy_research_test

0.3103448275862069

# 설문조사 전체 데이터셋 MBTI 예측

In [46]:
research_rev = research.copy()

In [47]:
pred_EI_researchalldata = dt_clf_EI_motivation.predict(X_EI_motivation)
pred_SN_researchalldata = cb_clf_SN_all.predict(X_SN_all)
pred_FT_researchalldata = rf_clf_FT.predict(X_FT)
pred_JP_researchalldata = cb_clf_JP_gender.predict(X_JP_gender)

research_rev['EI'] = pred_EI_researchalldata
research_rev['SN'] = pred_SN_researchalldata
research_rev['FT'] = pred_FT_researchalldata
research_rev['JP'] = pred_JP_researchalldata
research_rev['MBTI예측값'] = research_rev['EI'] + research_rev['SN'] + research_rev['FT'] + research_rev['JP']
research_rev.rename({'MBTI':'MBTI실제값'}, inplace=True, axis=1)
research_rev

,타임스탬프,성별,연령,거주지,MBTI실제값,자연/도시,숙박/당일,익숙한/새로운,편하지만비싼숙소/불편하지만저렴한숙소,휴양휴식/체험활동,...,좋았던국내여행지 2순위 시도,좋았던국내여행지 2순위 세부,좋았던국내여행지 3순위 시도,좋았던국내여행지 3순위 세부,이메일 주소,EI,SN,FT,JP,MBTI예측값
0,2024. 2. 7 오후 4:44:43,여자,20대,울산광역시,ENFP,후자 조금선호,전자 조금선호,후자 조금선호,전자 중간선호,후자 중간선호,...,서울특별시,서울특별시,경상북도,경주시,NaN,E,N,F,P,ENFP
1,2024. 2. 7 오후 4:45:59,여자,30대,충청남도,INFP,전자 중간선호,전자 조금선호,후자 매우선호,전자 중간선호,전자 조금선호,...,전라남도,여수시,경상남도,통영시,NaN,I,N,F,P,INFP
2,2024. 2. 7 오후 4:48:04,여자,20대,경기도,INTJ,전자 중간선호,전자 중간선호,후자 중간선호,전자 조금선호,후자 중간선호,...,제주특별자치도,서귀포시,부산광역시,부산광역시,NaN,I,N,T,P,INTP
3,2024. 2. 7 오후 4:48:08,여자,20대,울산광역시,ENFP,후자 조금선호,전자 조금선호,후자 조금선호,전자 중간선호,후자 중간선호,...,서울특별시,서울특별시,경상북도,경주시,NaN,E,N,F,P,ENFP
4,2024. 2. 7 오후 4:49:06,남자,20대,경기도,ENTP,전자 중간선호,전자 중간선호,후자 중간선호,전자 중간선호,전자 중간선호,...,강원도,양양군,NaN,NaN,NaN,E,N,T,P,ENTP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136,2024. 2. 8 오후 10:59:46,여자,20대,울산광역시,ISTJ,후자 중간선호,전자 중간선호,후자 중간선호,후자 조금선호,후자 중간선호,...,전라남도,순천,전라남도,담양,t2284@hanmail.net,I,S,T,J,ISTJ
137,2024. 2. 8 오후 11:40:24,여자,20대,경기도,ESTP,후자 중간선호,전자 중간선호,후자 중간선호,전자 조금선호,전자 조금선호,...,충청북도,충주시,경기도,가평군,lina_celebrity@naver.com,E,S,F,P,ESFP
138,2024. 2. 9 오전 4:59:28,남자,30대,전라북도,INFJ,전자 조금선호,전자 중간선호,전자 조금선호,전자 조금선호,전자 조금선호,...,경상남도,상주,NaN,NaN,dfvb8932@naver.com,I,N,F,J,INFJ
139,2024. 2. 10 오후 4:40:42,남자,20대,경기도,ISFJ,후자 중간선호,전자 매우선호,후자 중간선호,전자 중간선호,전자 매우선호,...,강원도,강릉시,강원도,춘천시,pohup@naver.com,I,S,F,J,ISFJ


In [48]:
research_rev_summary = research_rev[['MBTI실제값', 'MBTI예측값']]
research_rev_summary

,MBTI실제값,MBTI예측값
0,ENFP,ENFP
1,INFP,INFP
2,INTJ,INTP
3,ENFP,ENFP
4,ENTP,ENTP
...,...,...
136,ISTJ,ISTJ
137,ESTP,ESFP
138,INFJ,INFJ
139,ISFJ,ISFJ


In [49]:
accuracy_research_rev = accuracy_score(research_rev_summary['MBTI실제값'], research_rev_summary['MBTI예측값'])
accuracy_research_rev

0.8581560283687943